In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tqdm import tqdm
import argparse
import wandb
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
import rasterio
from model import *
from dataset import *
from utils import *
import sys
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import os
import glob
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
run = wandb.init(project="Gully_Classification")
artifact = run.use_artifact('tousi-team/Gully_Classification/model_epoch_100:v0', type='model')
artifact_dir = artifact.download("../weak-supervision/trained_models/")

wandb: Currently logged in as: stmmc (tousi-team). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model_epoch_100:v0, 114.00MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:11.5


In [3]:
class RGB_RasterTilesDataset_Geo(Dataset):
    def __init__(self, dem_dir, so_dir, rgb_dir, transform=None):
        self.dem_dir = dem_dir
        self.so_dir = so_dir
        self.rgb_dir = rgb_dir
        self.transform = transform
        # Assume all DEM, SO, and RGB files share the same tile identifiers
        self.tile_identifiers = [f.split('_')[-1].split('.')[0] for f in os.listdir(dem_dir) if 'dem_tile' in f]

    def __len__(self):
        return len(self.tile_identifiers)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tile_id = self.tile_identifiers[idx]
        dem_file = os.path.join(self.dem_dir, f'dem_tile_{tile_id}.tif')
        so_file = os.path.join(self.so_dir, f'dem_tile_{tile_id}.tif')
        rgb_files = [os.path.join(self.rgb_dir, f'rgb{k}_tile_{tile_id}.tif') for k in range(6)]

        # Prepare sample dictionary
        sample = {}

        # Read DEM file and extract the transform
        with rasterio.open(dem_file) as src:
            dem_image = src.read(1)  # Read the first band
            dem_transform = src.transform
            sample['DEM'] = dem_image
            sample['DEM_transform'] = dem_transform

        # Read SO file and extract the transform
        with rasterio.open(so_file) as src:
            so_image = src.read(1)
            so_transform = src.transform
            sample['SO'] = so_image
            sample['SO_transform'] = so_transform

        # Read RGB files and extract their transforms
        rgb_images = []
        rgb_transforms = []
        for file in rgb_files:
            with rasterio.open(file) as src:
                rgb_images.append(src.read([1, 2, 3]))  # Read the RGB bands
                rgb_transforms.append(src.transform)
        sample['RGB'] = rgb_images
        sample['RGB_transforms'] = rgb_transforms

        if self.transform:
            sample = self.transform(sample)

        return sample

In [4]:
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
from rasterio.transform import from_origin

def save_as_geotiff(data, profile, output_path, single_value=False, image_size=128):
    """
    Save a numpy array or single value as a GeoTIFF file with a given raster profile.
    
    :param data: Numpy array to save (2D or 3D array where 3D includes bands) or a single value.
    :param profile: Dictionary containing raster metadata like crs, transform, etc.
    :param output_path: Path where the GeoTIFF file will be saved
    """
    # Check if data is a single value and create a 2D array with the same value
    if single_value:

        data = np.full((image_size, image_size), data, dtype=profile.get('dtype', 'float32'))

    # Update the profile to accommodate the data dimensions
    profile.update({
        'dtype': data.dtype,
        'height': data.shape[0],
        'width': data.shape[1],
        'count': 1 if data.ndim == 2 else data.shape[0],  # Assuming bands are the first dimension if 3D
        'driver': 'GTiff',
        'nodata': None  # Set this to the appropriate nodata value if required
    })
#     print(profile)

    # Save the data to a GeoTIFF file
    with rasterio.open(output_path, 'w', **profile) as dst:
        if data.ndim == 2:
#             print('2')
            dst.write(data, 1)  # Write data as the first band
        else:
            for i in range(data.shape[0]):
                dst.write(data[i], i + 1)  # Write each band data
                
def calculate_stats(folder_path):
    means = []
    stds = []
    
    # Loop through all files in the directory
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".tif"):
            file_path = os.path.join(folder_path, filename)
            # Open the TIFF file
            with rasterio.open(file_path) as src:
                # Read data, assuming it's a single band
                array = src.read(1)
                # Combine conditions for NoData and zero values
                if src.nodata is not None:
                    mask = (array != src.nodata) & (array != 0)
                else:
                    mask = (array != 0)
                
                # Apply mask
                valid_data = array[mask]
                
                # Calculate mean and std dev and append to lists if valid data exists
                if valid_data.size > 0:
                    means.append(np.mean(valid_data))
                    stds.append(np.std(valid_data))
                else:
                    print(f"Warning: No valid data in file {filename} after masking. Skipping statistics.")

    # Calculate overall statistics
    overall_mean = np.mean(means) if means else 0
    overall_std = np.mean(stds) if stds else 0

    return overall_mean, overall_std

In [ ]:
field_name = 'Gotman'

In [6]:
# dem_dir = f'/home/macula/SMATousi/Gullies/SO_Paper/data/{field_name}/dem/'
# so_dir = f'/home/macula/SMATousi/Gullies/SO_Paper/data/{field_name}/dem/'
# rgb_dir = f'/home/macula/SMATousi/Gullies/SO_Paper/data/{field_name}/rgb/'
# pretrained_model_path = '/home/macula/SMATousi/cluster/docker-images/dem2so_more_data/pre_models/B3_rn50_moco_0099_ckpt.pth'

dem_dir = f'/home/macula/SMATousi/Gullies/ground_truth/organized_data/Google_Lines_Paper_Evaluation/downloaded_raw_1/dem/'
so_dir = f'/home/macula/SMATousi/Gullies/ground_truth/organized_data/Google_Lines_Paper_Evaluation/downloaded_raw_1/dem/'
rgb_dir = f'/home/macula/SMATousi/Gullies/ground_truth/organized_data/Google_Lines_Paper_Evaluation/downloaded_raw_1/rgb/'
# pretrained_model_path = '/home/macula/SMATousi/cluster/docker-images/dem2so_more_data/pre_models/B3_rn50_moco_0099_ckpt.pth'


mean, std = calculate_stats(dem_dir)
print("Overall Mean:", mean)
print("Overall Standard Deviation:", std)

class RGB_RasterTransform_Geo:
    """
    A custom transform class for raster data.
    """
    def __init__(self):
        pass

    def __call__(self, sample):
        dem, so, rgb = sample['DEM'], sample['SO'], sample['RGB']
        dem_meta, so_meta, rgb_meta = sample['DEM_transform'], sample['SO_transform'], sample['RGB_transforms']

        # Random horizontal flipping
        # if torch.rand(1) > 0.5:
        #     dem = TF.hflip(dem)
        #     so = TF.hflip(so)

        # # Random vertical flipping
        # if torch.rand(1) > 0.5:
        #     dem = TF.vflip(dem)
        #     so = TF.vflip(so)

        # Convert numpy arrays to tensors
        dem = TF.to_tensor(dem)
        so = TF.to_tensor(so)
        rgb_images = [TF.to_tensor(image) for image in rgb]
        float_rgb_images = [image.float() for image in rgb_images]
        # rgb_images = rgb_images.float()

        dem = TF.normalize(dem, mean, std)

        so = so.long()

        return {'DEM': dem, 'SO': so.squeeze(), 'RGB': float_rgb_images,
                'DEM_transform' : dem_meta, 'SO_transform' : so_meta, 'RGB_transforms' : rgb_meta}

transform = RGB_RasterTransform_Geo()
    
dataset = RGB_RasterTilesDataset_Geo(dem_dir=dem_dir, so_dir=so_dir, rgb_dir=rgb_dir, transform=transform)
# dataset = RGB_RasterTilesDataset_Geo(dem_dir=dem_dir, so_dir=so_dir, rgb_dir=rgb_dir)

batch_size = 1
learning_rate = 0.0001
epochs = 1
number_of_workers = 0
image_size = 128
val_percent = 0.0

n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, num_workers=number_of_workers, pin_memory=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=number_of_workers, pin_memory=True, drop_last=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

model = Gully_Classifier(input_size=6*2048, hidden_size=512, output_size=1).to(device)

# model = RGB_DEM_to_SO(resnet_output_size=(8, 8), 
#                             fusion_output_size=(128, 128), 
#                             model_choice = "Unet_1", 
#                             resnet_saved_model_path=pretrained_model_path,
#                             dropout_rate=0.5).to(device)

from torch.optim import Adam
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

state_dict = torch.load('./artifacts/models/model_epoch_100.pth')
# state_dict_new = torch.load('./artifacts/new_loss/model_epoch_600.pth')

new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

print(len(train_loader))

model.eval()

for i, batch in enumerate(tqdm(train_loader)):
# for i, batch in enumerate(train_loader):
    
    dem = batch['DEM'].to(device)
    so = batch['SO'].to(device)
    rgbs = [batch['RGB'][k].to(device) for k in range(6)]

    permute_rgbs = [torch.permute(image,(0,2,1,3)) for image in rgbs]

    
    output = model(permute_rgbs)
    
#     print(output)
    
#     break
    
#     val_outputs = model(dem, permute_rgbs)

#     pred = F.softmax(val_outputs, dim=1)              
#     pred = torch.argmax(pred, dim=1).squeeze(1)
    
    profile = {
    'transform': batch['SO_transform'],  # Example values: (west, north, xsize, ysize)
    'crs': 'EPSG:4326',  # Standard WGS84 CRS
    }

    # Path to save the GeoTIFF file
    root_path = f'/home/macula/SMATousi/Gullies/ground_truth/organized_data/tiled_HUCs/HUC_102702060102/classification_results'
    os.makedirs(root_path, exist_ok=True)
    output_file_path = f'{root_path}/{i}.tif'
    

    # Call the function to save the file
#     print(output.float().cpu().detach().numpy().squeeze())
    save_as_geotiff(output.float().cpu().detach().numpy().squeeze(), profile, output_file_path, single_value=True)
    
#     break


Overall Mean: 261.96936
Overall Standard Deviation: 3.940066
cuda


/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


143


  0%|                                                                                                                    | 0/143 [00:00<?, ?it/s]


RasterioIOError: /home/macula/SMATousi/Gullies/ground_truth/organized_data/Google_Lines_Paper_Evaluation/downloaded_raw_1/rgb/rgb0_tile_009.tif: No such file or directory

wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)


In [ ]:
profile

In [5]:

# input_folder = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/tiled_HUCs/HUC_102702060102/classification_results'  # Update this path to your folder path
# output_filepath = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/tiled_HUCs/HUC_102702060102/merged_classification_results.tif'  # Set your output file path

# # Search for TIFF files in the folder
# search_criteria = "*.tif"
# query = os.path.join(input_folder, search_criteria)
# tif_files = glob.glob(query)

# # Function to yield open datasets one by one
# def open_tiff_files(filepaths):
#     for filepath in filepaths:
#         with rasterio.open(filepath) as src:
#             yield src.read(1), src.transform

# # List to hold the datasets and transformations
# datasets = []
# transforms = []

# Open and append each TIFF file's data and transform to the lists
# for data, transform in open_tiff_files(tif_files):
#     datasets.append(data)
#     transforms.append(transform)

# Merge function from rasterio
mosaic, out_trans = merge(datasets)

# Copy the metadata from one of the source files
with rasterio.open(tif_files[0]) as src:
    out_meta = src.meta.copy()

# Update the metadata to reflect the number of layers
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans,
    "crs": src.crs
})

# Write the mosaic raster to the new file
with rasterio.open(output_filepath, "w", **out_meta) as dest:
    dest.write(mosaic, 1)

print("Merge completed successfully. Output saved at:", output_filepath)

AttributeError: 'numpy.ndarray' object has no attribute 'profile'

In [ ]:
  src_files_to_mosaic

[array([[0.00474909, 0.00474909, 0.00474909, ..., 0.00474909, 0.00474909,
         0.00474909],
        [0.00474909, 0.00474909, 0.00474909, ..., 0.00474909, 0.00474909,
         0.00474909],
        [0.00474909, 0.00474909, 0.00474909, ..., 0.00474909, 0.00474909,
         0.00474909],
        ...,
        [0.00474909, 0.00474909, 0.00474909, ..., 0.00474909, 0.00474909,
         0.00474909],
        [0.00474909, 0.00474909, 0.00474909, ..., 0.00474909, 0.00474909,
         0.00474909],
        [0.00474909, 0.00474909, 0.00474909, ..., 0.00474909, 0.00474909,
         0.00474909]], dtype=float32),
 array([[0.00012244, 0.00012244, 0.00012244, ..., 0.00012244, 0.00012244,
         0.00012244],
        [0.00012244, 0.00012244, 0.00012244, ..., 0.00012244, 0.00012244,
         0.00012244],
        [0.00012244, 0.00012244, 0.00012244, ..., 0.00012244, 0.00012244,
         0.00012244],
        ...,
        [0.00012244, 0.00012244, 0.00012244, ..., 0.00012244, 0.00012244,
         0.00012244